## Exercise 1 Representations

__Binary:__ 
<br>
On the computer. Spices example. Each gene represents one spice, and the value can be true or false.
<br>

Mutation: Bit-flip. With a probability $p_m$ flip one, or several, digits.
<br>

Recombination:  L#2 s. 32-: Single-point crossover, n-point crossover, uniform crossover (Tail = parent 1, heads = parent 2, flip through each gene. Child 2 opposite of child 1.)
<br>

__Integer:__ 
<br>
TSP problem, 8 queens chess problem. Vector of different integers, <mark> DIP, colors (red, blue)
<br>
Mutation: Creeping (slow) or resetting (larger change)
<br>
Crossover: N-point, uniform.

__Floating point numbers:__ 
<br>
Maximization of numbers smaller than integers.
<br>
Mutation: Unform mutation, same role as bit flip for bindary or random resetting for integers. Draw from [LB, UB], upper lower bound. 
<br>
Non-uniform mutation: Use gaussian each variable seprately.
<br>
Crossover: Discrete recombination: Value always from 1 of parents (No averages). n-point crossover or uniform.  
<br>
Intermediate recombination: Children, weighted average of parents, as I did in my 1st EA.
<br>
Example: Simple arithmetic crossover: Crossover + intermediate after crossover point.
<br>
Example: Single arithmetic crossover. Child 1 equal parent 1 to the left of point, at point a weighted average, and parent 2 to the right of point.
<br>
Example: Whole arithmetic crossover: weighted average everywhere.
<br>

__Trees:__ Loops, mathematical expressions.
<br>
Mutation: replace randomly chosen subtree by randomly generated tree
<br>
Crossover:Most common recombination: exchange two randomly chosen subtrees among the parents
<br>

__Permutation:__ 
<br>
Travelling sales person. One number only appears once. List of n different integers.
<br>
Mutation:
    <br>
    Must change at least 2 values. Swapping. Insert mutation: Move one next to another and change all others to come next in same order as they were. Scramble mutation: for certain number of genes randomly reorder these genes. Inversion: For a chosen area, reverse the order of the genes.
 <br>
Crossover:
<br>
   Adjacance important (who is neighbor). TSP. --> Conserve adjacancy.
   <br>
   Partially mapped crossover and edge recombination: L#2 s. 52
   <br>
   Order importan others. Need to conserve order. Order corssover and cycle crossover.
   <br>
   Partially mapped crossover (PMX). See slides L#2 s 56.
   <br>
   Edge recombination. See slide.
   <br>
   Order crossover: L#2 s 61. Understandable example.
   <br>
   Cycle crossover: L#2 s 62. 
